In [15]:
import traci
import os, sys
import sumolib
import pandas as pd

import xml.etree.ElementTree as ET


In [2]:
if 'SUMO_HOME' in os.environ:
    tools = os.path.join(os.environ['SUMO_HOME'], 'tools')
    sys.path.append(tools)
else:   
    sys.exit("please declare environment variable 'SUMO_HOME'")

In [3]:
# parse the net
net = sumolib.net.readNet('Chattanooga_SUMO_Network.net.xml')

In [8]:
# Connect to SUMO  

traci.start(["sumo", "-n", "Chattanooga_SUMO_Network.net.xml"])

#Get geographic coordinates of bus stops

Stops = pd.read_excel('odMatrix.xlsx',sheet_name = "odMatrix").copy()
Pickup_LAT = Stops['pickup_lat']
Pickup_LON = Stops['pickup_lon']

#Convert geographic coordinates to position info (edge ID, lane position and lane index)
def get_edge(lon, lat):
    edgeID, lanePosition, laneIndex = traci.simulation.convertRoad(lon,lat, True)
    return edgeID, lanePosition, laneIndex

edgeinf =[]
for i in range(len(Pickup_LAT)):
    edgeinf.append(get_edge(Pickup_LON[i],Pickup_LAT[i]))
# Write to     
df = pd.DataFrame(edgeinf, columns=['edgeID', 'lanepos', 'laneind'])
df.to_excel(r'origin.xlsx', index = False)
traci.close()

In [10]:
# Connect to SUMO  

traci.start(["sumo", "-n", "Chattanooga_SUMO_Network.net.xml"])

#Get geographic coordinates of bus stops

Stops = pd.read_excel('odMatrix.xlsx',sheet_name = "odMatrix").copy()
Pickup_LAT = Stops['dropoff_lat']
Pickup_LON = Stops['dropoff_lon']

#Convert geographic coordinates to position info (edge ID, lane position and lane index)
def get_edge(lon, lat):
    edgeID, lanePosition, laneIndex = traci.simulation.convertRoad(lon,lat, True)
    return edgeID, lanePosition, laneIndex

edgeinf =[]
for i in range(len(Pickup_LAT)):
    edgeinf.append(get_edge(Pickup_LON[i],Pickup_LAT[i]))
# Write to     
df = pd.DataFrame(edgeinf, columns=['edgeID', 'lanepos', 'laneind'])
df.to_excel(r'destination.xlsx', index = False)
traci.close()

In [19]:

# Read the Excel file into a pandas DataFrame
df = pd.read_excel('odMatrix.xlsx')

# Create the root element of the XML tree
root = ET.Element("routes", attrib={"xmlns:xsi": "http://www.w3.org/2001/XMLSchema-instance",
                                   "xsi:noNamespaceSchemaLocation": "http://sumo.dlr.de/xsd/routes_file.xsd"})

# Iterate over the rows in the DataFrame
for index, row in df.iterrows():
    # Create a trip element for each row
    trip = ET.SubElement(root, "trip", attrib={"id": "t{}".format(index+1),
                                               "depart": str(row['pickup_time_secs']),
                                               "from": str(row['O_edgeID']),
                                               "to": str(row['D_edgeID']),
                                               "type": "passenger",
                                               "departLane": "best",
                                               "departPos": "free",
                                               "departSpeed": "max"})
    
    # Add indentation before the trip element
    trip.text = "\n" + " " * 3

# Create the XML tree
tree = ET.ElementTree(root)

# Set the indent level to 3 spaces
ET.indent(tree, space=" ", level=3)

# Write the XML tree to a file
tree.write("output.xml", encoding="utf-8", xml_declaration=True)




duarouter --route-files trips.xml -n  Chattanooga_SUMO_Network.net.xml --unsorted-input  --ptline-routing --output-file trip.rou.xml --ignore-errors

DUA

python3 duaIterate.py --net-file Chattanooga_SUMO_Network.net.xml -t trip.rou.xml -l 2 -C -m   -E -L --time-to-teleport 50 --clean-alt 